<a href="https://colab.research.google.com/github/david-siqi-liu/cs684-final-project/blob/master/DANN_Colab_CPU_Project_14Dec_21_34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Download

In [0]:
# Project

from urllib.request import urlretrieve
import os
from zipfile import ZipFile

def download(url, file):
    if not os.path.isfile(file):
        urlretrieve(url, file)

#If the downloaded file is a zip file than you can use below function to unzip it.
def uncompress_features_labels(source, file, dest):
    if not os.path.isdir(file):
        with ZipFile(source) as zipf:
            zipf.extractall(dest)

            

In [349]:
# Project
# Source images
for file in ['infograph.zip', 'quickdraw.zip', 'real.zip', 'sketch.zip']:
  download('http://csr.bu.edu/ftp/visda/2019/multi-source/' + file,
           file)
  print("Downloaded: {0}".format(file))
  uncompress_features_labels(file,
                             'data/' + file.split('.zip')[0],
                             'data/')
  print("Extracted: {0}".format(file))

Downloaded: infograph.zip
Extracted: infograph.zip
Downloaded: quickdraw.zip
Extracted: quickdraw.zip
Downloaded: real.zip
Extracted: real.zip
Downloaded: sketch.zip
Extracted: sketch.zip


In [350]:
# Project
# Target images (labelled)
download('http://csr.bu.edu/ftp/visda/2019/multi-source/groundtruth/clipart.zip', 'clipart.zip')
print("Downloaded: clipart.zip")
uncompress_features_labels('clipart.zip', 'data/clipart', 'data/')
print("Extracted: clipart.zip")

Downloaded: clipart.zip
Extracted: clipart.zip


In [351]:
!mkdir label
# Source labels
for file in ['infograph', 'quickdraw', 'real', 'sketch']:
  download('http://csr.bu.edu/ftp/visda/2019/multi-source/txt/' + file + '_train.txt',
           'label/' + file + '_train.txt')
  download('http://csr.bu.edu/ftp/visda/2019/multi-source/txt/' + file + '_test.txt',
           'label/' + file + '_test.txt')
  
# Target labels (groundtruth)
download('http://csr.bu.edu/ftp/visda/2019/multi-source/groundtruth/txt/clipart_test.txt',
           'label/clipart_test.txt')

mkdir: cannot create directory ‘label’: File exists


# Library


In [0]:
import random
import os
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torch.utils.data as data
import torch.nn as nn
import numpy as np
from torch.autograd import Function
from torchvision import datasets
from torchvision import transforms
from PIL import Image, ImageColor

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Parameters

In [353]:
# Original
source_dataset_name = 'source'
target_dataset_name = 'target'
model_root = 'models'
!mkdir models
cuda = False
cudnn.benchmark = True
lr = 1e-3
batch_size = 32
image_size = 28
n_epoch = 5

manual_seed = random.randint(1, 10000)
random.seed(manual_seed)
torch.manual_seed(manual_seed)


mkdir: cannot create directory ‘models’: File exists


# Data Loader

In [354]:
num_labels = 10

np.random.seed(647)

# Randomly select labels

selected_original_labels = random.sample(range(344), num_labels)

selected_original_labels.sort()

print("Selected labels: {0}".format(selected_original_labels))

# Mapping
label_to_original_mapping = {}
for i in range(0, num_labels):
  label_to_original_mapping[i] = selected_original_labels[i]

original_to_label_mapping = {v : k for k, v in label_to_original_mapping.items()}
print("Original to label mapping: {0}".format(original_to_label_mapping))
print("Label to original mapping: {0}".format(label_to_original_mapping))

# Find English term
original_to_english_mapping = {}
for line in open("label/clipart_test.txt"):
  d = line.strip().split(' ')
  l = int(d[1])
  original_to_english_mapping[l] = d[0].split("/")[1]

print("Original to English mapping: {0}".format(original_to_english_mapping))

# Final label mapping
labels = {}
for i in range(0, num_labels):
  labels[i] = original_to_english_mapping[label_to_original_mapping[i]]

print("Labels: {0}".format(labels))

Selected labels: [1, 29, 82, 108, 123, 160, 179, 211, 213, 327]
Original to label mapping: {1: 0, 29: 1, 82: 2, 108: 3, 123: 4, 160: 5, 179: 6, 211: 7, 213: 8, 327: 9}
Label to original mapping: {0: 1, 1: 29, 2: 82, 3: 108, 4: 123, 5: 160, 6: 179, 7: 211, 8: 213, 9: 327}
Original to English mapping: {0: 'aircraft_carrier', 1: 'airplane', 2: 'alarm_clock', 3: 'ambulance', 4: 'angel', 5: 'animal_migration', 6: 'ant', 7: 'anvil', 8: 'apple', 9: 'arm', 10: 'asparagus', 11: 'axe', 12: 'backpack', 13: 'banana', 14: 'bandage', 15: 'barn', 16: 'baseball', 17: 'baseball_bat', 18: 'basket', 19: 'basketball', 20: 'bat', 21: 'bathtub', 22: 'beach', 23: 'bear', 24: 'beard', 25: 'bed', 26: 'bee', 27: 'belt', 28: 'bench', 29: 'bicycle', 30: 'binoculars', 31: 'bird', 32: 'birthday_cake', 33: 'blackberry', 34: 'blueberry', 35: 'book', 36: 'boomerang', 37: 'bottlecap', 38: 'bowtie', 39: 'bracelet', 40: 'brain', 41: 'bread', 42: 'bridge', 43: 'broccoli', 44: 'broom', 45: 'bucket', 46: 'bulldozer', 47: 'b

In [0]:
###### LIST LOADER #######

def default_loader(path):
    """Default loader
    """
    return Image.open(path).convert('RGB')

def collect_images(img_dir, labels):
  """Return a list of (image path, label)

  Parameters:
  img_dir (String) : the directory containing the images
  labels (List[String]) : a list of labels (merged multiple sources together)

  Returns:
  List[(String, String)]
  """
  images = []
  indices = {}
  pos = 0
  for line in labels:
    data = line.strip().split(' ')
    path = os.path.join(img_dir, data[0])
    original_label = int(data[1])
    # Among the 10 labels selected
    if original_label in selected_original_labels:
      label = original_to_label_mapping[original_label]
      item = (path, label)
      images.append(item)
      if label not in indices.keys():
        indices[label] = [pos]
      else:
        indices[label].append(pos)
      pos += 1
  return images, indices

class MyDataset(data.Dataset):
    """ Custom class for loading image list
    """
    def __init__(self, img_dir, labels, transform=None, loader=default_loader):
        imgs, indices = collect_images(img_dir, labels)
        self.img_dir = img_dir
        self.imgs = imgs
        self.indices = indices
        self.num_imgs = len(imgs)
        self.transform = transform
        self.loader = loader

    def __getitem__(self, index):
        path, label = self.imgs[index]
        img = self.loader(path)

        if self.transform is not None:
            img = self.transform(img)
            
        return img, label

    def __len__(self):
        return len(self.imgs)

    def get_random_sample_index(self, label, n=1):
      return np.random.choice(self.indices[label], n)[0]

      

In [0]:
###### IMAGE LOADER #######

def make_data_set(img_dir, label_dir, domain_list, transforms, train_or_test):
  labels = []
  for domain in domain_list:
    # E.g. "label/infograph_test.txt"
    label_file = label_dir + domain + '_' + train_or_test + '.txt'
    for line in open(label_file):
      labels.append(line)
  return MyDataset(img_dir, labels, transforms)

def make_data_loader(dataset, train_or_test):
  if (train_or_test == 'train'):
    return data.DataLoader(dataset, batch_size=4,
                           shuffle=True, num_workers=4)
  else:
    return data.DataLoader(dataset, batch_size=4,
                           shuffle=True, num_workers=4)
    



In [0]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

def tensor_to_PIL(img, mean, std):
  img = img.numpy().transpose((1, 2, 0))
  img = img * std + mean
  img = np.clip(img, 0, 1)
  return img

In [0]:
# # Original
# class GetLoader(data.Dataset):
#     def __init__(self, data_root, data_list, transform=None):
#         self.root = data_root
#         self.transform = transform

#         f = open(data_list, 'r')
#         data_list = f.readlines()
#         f.close()

#         self.n_data = len(data_list)

#         self.img_paths = []
#         self.img_labels = []

#         for data in data_list:
#             self.img_paths.append(data[:-3])
#             self.img_labels.append(data[-2])

#     def __getitem__(self, item):
#         img_paths, labels = self.img_paths[item], self.img_labels[item]
#         imgs = Image.open(os.path.join(self.root, img_paths)).convert('RGB')

#         if self.transform is not None:
#             imgs = self.transform(imgs)
#             labels = int(labels)

#         return imgs, labels

#     def __len__(self):
#         return self.n_data


# Load Data

In [0]:
# # Original
# img_transform_source = transforms.Compose([
#     transforms.Resize(image_size),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=(0.1307,), std=(0.3081,))
# ])

# img_transform_target = transforms.Compose([
#     transforms.Resize(image_size),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
# ])

# dataset_source = datasets.MNIST(
#     root='dataset',
#     train=True,
#     transform=img_transform_source,
#     download=True
# )

# dataloader_source = torch.utils.data.DataLoader(
#     dataset=dataset_source,
#     batch_size=batch_size,
#     shuffle=True,
#     num_workers=0)

# train_list = os.path.join(target_image_root, 'mnist_m_train_labels.txt')

# dataset_target = GetLoader(
#     data_root=os.path.join(target_image_root, 'mnist_m_train'),
#     data_list=train_list,
#     transform=img_transform_target
# )

# dataloader_target = torch.utils.data.DataLoader(
#     dataset=dataset_target,
#     batch_size=batch_size,
#     shuffle=True,
#     num_workers=0)



In [0]:
home_dir = ''
img_dir = home_dir + 'data/'
label_dir = home_dir + 'label/'
# source = ['quickdraw', 'real', 'sketch']
# target = ['clipart']
source = ['real', 'sketch']
target = ['clipart']


train_transforms = transforms.Compose([transforms.Resize([28, 28]),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize([28, 28]),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

target_transforms = transforms.Compose([transforms.Resize([28, 28]),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406],
                                                             [0.229, 0.224, 0.225])])

dataset_source = make_data_set(img_dir, label_dir, source, train_transforms, 'train')
dataloader_source = make_data_loader(dataset_source, 'train')
#source_test_dataset = make_data_set(img_dir, label_dir, source, test_transforms, 'test')
#source_test_dataloader = make_data_loader(source_test_dataset, 'test')
dataset_target = make_data_set(img_dir, label_dir, target, target_transforms, 'test')
dataloader_target = make_data_loader(dataset_target, 'test')

In [359]:
print("Source train dataset size: {0}".format(len(dataset_source)))
#print("Source test dataset size: {0}".format(len(source_test_dataset)))
print("Target dataset size: {0}".format(len(dataset_target)))

Source train dataset size: 5250
Target dataset size: 214


# ReverseLayerF

In [0]:

class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None



# Model


In [0]:
class CNNModel(nn.Module):

    def __init__(self):
        super(CNNModel, self).__init__()
        self.feature = nn.Sequential()
        self.feature.add_module('f_conv1', nn.Conv2d(3, 64, kernel_size=5))
        self.feature.add_module('f_bn1', nn.BatchNorm2d(64))
        self.feature.add_module('f_pool1', nn.MaxPool2d(2))
        self.feature.add_module('f_relu1', nn.ReLU(True))
        self.feature.add_module('f_conv2', nn.Conv2d(64, 50, kernel_size=5))
        self.feature.add_module('f_bn2', nn.BatchNorm2d(50))
        self.feature.add_module('f_drop1', nn.Dropout2d())
        self.feature.add_module('f_pool2', nn.MaxPool2d(2))
        self.feature.add_module('f_relu2', nn.ReLU(True))

        self.class_classifier = nn.Sequential()
        self.class_classifier.add_module('c_fc1', nn.Linear(50 * 4 * 4, 100))
        self.class_classifier.add_module('c_bn1', nn.BatchNorm1d(100))
        self.class_classifier.add_module('c_relu1', nn.ReLU(True))
        self.class_classifier.add_module('c_drop1', nn.Dropout2d())
        self.class_classifier.add_module('c_fc2', nn.Linear(100, 100))
        self.class_classifier.add_module('c_bn2', nn.BatchNorm1d(100))
        self.class_classifier.add_module('c_relu2', nn.ReLU(True))
        self.class_classifier.add_module('c_fc3', nn.Linear(100, 10))
        self.class_classifier.add_module('c_softmax', nn.LogSoftmax())

        self.domain_classifier = nn.Sequential()
        self.domain_classifier.add_module('d_fc1', nn.Linear(50 * 4 * 4, 100))
        self.domain_classifier.add_module('d_bn1', nn.BatchNorm1d(100))
        self.domain_classifier.add_module('d_relu1', nn.ReLU(True))
        self.domain_classifier.add_module('d_fc2', nn.Linear(100, 2))
        self.domain_classifier.add_module('d_softmax', nn.LogSoftmax(dim=1))

    def forward(self, input_data, alpha):
        input_data = input_data.expand(input_data.data.shape[0], 3, 28, 28)
        feature = self.feature(input_data)
        feature = feature.view(-1, 50 * 4 * 4)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.class_classifier(feature)
        domain_output = self.domain_classifier(reverse_feature)

        return class_output, domain_output

# Load Model

In [0]:

my_net = CNNModel()

# Setup Optimizer

In [0]:
optimizer = optim.Adam(my_net.parameters(), lr=lr)

loss_class = torch.nn.NLLLoss()
loss_domain = torch.nn.NLLLoss()

if cuda:
    my_net = my_net.cuda()
    loss_class = loss_class.cuda()
    loss_domain = loss_domain.cuda()

for p in my_net.parameters():
    p.requires_grad = True


# Test

In [0]:

def test(dataset_name, epoch):
    assert dataset_name in ['source', 'target']

    model_root = 'models'
   #image_root = os.path.join('dataset', dataset_name)

    cuda = False
    cudnn.benchmark = True
    batch_size = 32
    image_size = 28
    alpha = 0

    """load data"""

    home_dir = ''
    img_dir = home_dir + 'data/'
    label_dir = home_dir + 'label/'
    source = ['quickdraw', 'real', 'sketch']
    target = ['clipart']
    

    train_transforms = transforms.Compose([transforms.Resize([28, 28]),
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485, 0.456, 0.406],
                                                                [0.229, 0.224, 0.225])])

    test_transforms = transforms.Compose([transforms.Resize([28, 28]),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406],
                                                               [0.229, 0.224, 0.225])])

    target_transforms = transforms.Compose([transforms.Resize([28, 28]),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406],
                                                                 [0.229, 0.224, 0.225])])

    dataset_source = make_data_set(img_dir, label_dir, source, train_transforms, 'train')
    dataloader_source = make_data_loader(dataset_source, 'train')
    #source_test_dataset = make_data_set(img_dir, label_dir, source, test_transforms, 'test')
    #source_test_dataloader = make_data_loader(source_test_dataset, 'test')
    dataset_target = make_data_set(img_dir, label_dir, target, target_transforms, 'test')
    dataloader_target = make_data_loader(dataset_target, 'test')

    if dataset_name == 'source':
        dataset_source = make_data_set(img_dir, label_dir, source, train_transforms, 'train')
        dataloader = make_data_loader(dataset_source, 'train')
    else:
        dataset_target = make_data_set(img_dir, label_dir, target, target_transforms, 'test')
        dataloader = make_data_loader(dataset_target, 'test')


    """ training """

    model = CNNModel()
    checkpoint = torch.load('./models/mnist_mnistm_model_epoch_' + str(epoch) + '.pth')
    model.load_state_dict(checkpoint['state_dict'])
  
  
  
    my_net = model.eval()

    if cuda:
        my_net = my_net.cuda()

    len_dataloader = len(dataloader)
    data_target_iter = iter(dataloader)

    i = 0
    n_total = 0
    n_correct = 0

    while i < len_dataloader:

        # test model using target data
        data_target = data_target_iter.next()
        t_img, t_label = data_target

        batch_size = len(t_label)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        class_label = torch.LongTensor(batch_size)

        if cuda:
            t_img = t_img.cuda()
            t_label = t_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)
        class_label.resize_as_(t_label).copy_(t_label)

        class_output, _ = my_net(input_data=input_img, alpha=alpha)
        pred = class_output.data.max(1, keepdim=True)[1]
        n_correct += pred.eq(class_label.data.view_as(pred)).cpu().sum()
        n_total += batch_size

        i += 1

    accu = n_correct.data.numpy() * 1.0 / n_total
    
    print ('    epoch: %d, accuracy of the %s dataset: %f' % (epoch, dataset_name, accu))


# Training & Results

In [365]:

for epoch in range(n_epoch):

    len_dataloader = min(len(dataloader_source), len(dataloader_target))
    data_source_iter = iter(dataloader_source)
    data_target_iter = iter(dataloader_target)

    i = 0
    while i < len_dataloader:

        p = float(i + epoch * len_dataloader) / n_epoch / len_dataloader
        alpha = 2. / (1. + np.exp(-10 * p)) - 1

        # training model using source data
        data_source = data_source_iter.next()
        s_img, s_label = data_source

        my_net.zero_grad()
        batch_size = len(s_label)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        class_label = torch.LongTensor(batch_size)
        domain_label = torch.zeros(batch_size)
        domain_label = domain_label.long()

        if cuda:
            s_img = s_img.cuda()
            s_label = s_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(s_img).copy_(s_img)
        class_label.resize_as_(s_label).copy_(s_label)

        class_output, domain_output = my_net(input_data=input_img, alpha=alpha)
        err_s_label = loss_class(class_output, class_label)
        err_s_domain = loss_domain(domain_output, domain_label)

        # training model using target data
        data_target = data_target_iter.next()
        t_img, _ = data_target

        batch_size = len(t_img)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        domain_label = torch.ones(batch_size)
        domain_label = domain_label.long()

        if cuda:
            t_img = t_img.cuda()
            input_img = input_img.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)

        _, domain_output = my_net(input_data=input_img, alpha=alpha)
        err_t_domain = loss_domain(domain_output, domain_label)
        err = err_t_domain + err_s_domain + err_s_label
        err.backward()
        optimizer.step()

        i += 1

        print ("\r",'epoch: %d, [iter: %d / all %d], err_s_label: %f, err_s_domain: %f, err_t_domain: %f' \
              % (epoch, i, len_dataloader, err_s_label.data.cpu().numpy(),
                 err_s_domain.data.cpu().numpy(), err_t_domain.data.cpu().item()),end="")
    print(' ')
    
    torch.save({'state_dict': my_net.state_dict()}, '{0}/mnist_mnistm_model_epoch_{1}.pth'.format(model_root, epoch))


    test(source_dataset_name, epoch)
    test(target_dataset_name, epoch)

print('done')


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


 epoch: 0, [iter: 54 / all 54], err_s_label: 2.605547, err_s_domain: 0.717916, err_t_domain: 0.682170 
    epoch: 0, accuracy of the source dataset: 0.112343
    epoch: 0, accuracy of the target dataset: 0.074766
 epoch: 1, [iter: 54 / all 54], err_s_label: 2.538148, err_s_domain: 0.707359, err_t_domain: 0.678090 
    epoch: 1, accuracy of the source dataset: 0.184114
    epoch: 1, accuracy of the target dataset: 0.172897
 epoch: 2, [iter: 54 / all 54], err_s_label: 2.171398, err_s_domain: 0.700501, err_t_domain: 0.691629 
    epoch: 2, accuracy of the source dataset: 0.175429
    epoch: 2, accuracy of the target dataset: 0.210280
 epoch: 3, [iter: 54 / all 54], err_s_label: 1.976601, err_s_domain: 0.699192, err_t_domain: 0.724955 
    epoch: 3, accuracy of the source dataset: 0.158514
    epoch: 3, accuracy of the target dataset: 0.144860
 epoch: 4, [iter: 54 / all 54], err_s_label: 2.642530, err_s_domain: 0.682148, err_t_domain: 0.696405 
    epoch: 4, accuracy of the source dataset: